In [1]:
import pandas as pd

In [2]:
def calc_team_tourney_scores(df_teams, df_tourney, tourney_name):
    """
        Calculate the team's score in every tourney
    """
    # To scale score from OT
    df_tourney['WTeamID'].astype(dtype='int')
    df_tourney['LTeamID'].astype(dtype='int')
    df_tourney['WScore'] = df_tourney[['WScore', 'NumOT']].apply(lambda x: x[0] * 4.0 / (x[1] + 4), axis=1)
    df_tourney['LScore'] = df_tourney[['LScore', 'NumOT']].apply(lambda x: x[0] * 4.0 / (x[1] + 4), axis=1)
    
    # Team's win or lose mean score
    df_team_win_mean_score = df_tourney.groupby(['WTeamID'])['WScore'].mean().reset_index().rename(columns={'WTeamID': 'team_id', 'WScore': 'team_score'})
    df_team_lose_mean_score = df_tourney.groupby(['LTeamID'])['LScore'].mean().reset_index().rename(columns={'LTeamID': 'team_id', 'LScore': 'team_score'})
    # Team's mean score
    df_team_mean_score_concat = pd.concat([df_team_win_mean_score, df_team_lose_mean_score])
    df_team_mean_score = df_team_mean_score_concat.groupby('team_id')['team_score'].mean().reset_index()
    
    # Team's win or lose max score
    df_team_win_max_score = df_tourney.groupby(['WTeamID'])['WScore'].max().reset_index().rename(columns={'WTeamID': 'team_id', 'WScore': 'team_score'})
    df_team_lose_max_score = df_tourney.groupby(['LTeamID'])['LScore'].max().reset_index().rename(columns={'LTeamID': 'team_id', 'LScore': 'team_score'})
    # Team's max score
    df_team_max_score_concat = pd.concat([df_team_win_max_score, df_team_lose_max_score])
    df_team_max_score = df_team_max_score_concat.groupby('team_id')['team_score'].max().reset_index()
    
    # Team's win or lose min score
    df_team_win_min_score = df_tourney.groupby(['WTeamID'])['WScore'].min().reset_index().rename(columns={'WTeamID': 'team_id', 'WScore': 'team_score'})
    df_team_lose_min_score = df_tourney.groupby(['LTeamID'])['LScore'].min().reset_index().rename(columns={'LTeamID': 'team_id', 'LScore': 'team_score'})
    # Team's min score
    df_team_min_score_concat = pd.concat([df_team_win_min_score, df_team_lose_min_score])
    df_team_min_score = df_team_min_score_concat.groupby('team_id')['team_score'].min().reset_index()
    
    # Set index to team_id
    df_team_win_mean_score = df_team_win_mean_score.set_index('team_id')
    df_team_lose_mean_score = df_team_lose_mean_score.set_index('team_id')
    df_team_mean_score = df_team_mean_score.set_index('team_id')
    df_team_win_max_score = df_team_win_max_score.set_index('team_id')
    df_team_lose_max_score = df_team_lose_max_score.set_index('team_id')
    df_team_max_score = df_team_max_score.set_index('team_id')
    df_team_win_min_score = df_team_win_min_score.set_index('team_id')
    df_team_lose_min_score = df_team_lose_min_score.set_index('team_id')
    df_team_min_score = df_team_min_score.set_index('team_id')
    
    # Join all features
    df_tmp_teams = df_teams.join(df_team_win_mean_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_win_mean_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_lose_mean_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_lose_mean_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_mean_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_mean_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_win_max_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_win_max_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_lose_max_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_lose_max_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_max_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_max_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_win_min_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_win_min_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_lose_min_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_lose_min_score'}).set_index('team_id')
    df_tmp_teams = df_tmp_teams.join(df_team_min_score).reset_index().fillna(0).rename(columns={'team_score': tourney_name + '_min_score'}).set_index('team_id')
    
    return df_tmp_teams.reset_index()

In [8]:
# Load data
df_teams = pd.read_csv('./dataset/DataFiles/Teams.csv')
df_nacc_tourney_compact_results = pd.read_csv('./dataset/DataFiles/NCAATourneyCompactResults.csv')
df_regular_season_compact_results = pd.read_csv('./dataset/DataFiles/RegularSeasonCompactResults.csv')

# Treat teams data
df_teams_rename_colums = {'TeamID': 'team_id'}
df_teams_drop_columns = ['TeamName', 'FirstD1Season', 'LastD1Season']
df_teams = df_teams.rename(columns=df_teams_rename_colums)
df_teams = df_teams.drop(df_teams_drop_columns, axis=1)
df_teams = df_teams.astype(dtype='int')
df_teams = df_teams.set_index('team_id')

# Treat data
df_teams_ncaa = calc_team_tourney_scores(df_teams, df_nacc_tourney_compact_results, 'ncaa')
df_teams_regular = calc_team_tourney_scores(df_teams, df_regular_season_compact_results, 'regular')

In [9]:
df_teams_ncaa = df_teams_ncaa.set_index('team_id')
df_teams_regular = df_teams_regular.set_index('team_id')
df_train_teams_features = df_teams_ncaa.join(df_teams_regular, how='inner').reset_index()

In [10]:
df_train_teams_features.head()

364


,team_id,ncaa_win_mean_score,ncaa_lose_mean_score,ncaa_mean_score,ncaa_win_max_score,ncaa_lose_max_score,ncaa_max_score,ncaa_win_min_score,ncaa_lose_min_score,ncaa_min_score,regular_win_mean_score,regular_lose_mean_score,regular_mean_score,regular_win_max_score,regular_lose_max_score,regular_max_score,regular_win_min_score,regular_lose_min_score,regular_min_score
0,1101,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,75.703704,59.947748,67.825726,95.0,81.0,95.0,56.0,27.0,27.0
1,1102,0.000000,60.500000,60.500000,0.0,69.0,69.0,0.0,52.0,52.0,69.645410,59.056798,64.351104,111.0,89.0,111.0,41.6,33.0,33.0
2,1103,0.000000,56.500000,56.500000,0.0,64.0,64.0,0.0,42.0,42.0,75.157397,63.506500,69.331949,109.0,111.0,111.0,44.0,30.0,30.0
3,1104,78.322222,64.933333,71.627778,101.0,84.0,101.0,50.0,52.0,50.0,75.235805,63.598797,69.417301,114.0,91.0,114.0,42.0,37.0,37.0
4,1105,0.000000,69.000000,69.000000,0.0,69.0,69.0,0.0,69.0,69.0,74.188214,60.686747,67.437481,114.0,99.0,114.0,51.0,34.0,34.0


In [14]:
df_train_teams_features.to_csv('./features/teams_scores_features.csv', index=False)